In [1]:
# -*- coding: utf-8 -*-
# -----------------------------------------------------------
# Lake Fire (example): explicit transition overrides first,
# then modal+climate fallback for remaining inside-scar pixels,
# and write a remapped raster (LF2024 updated inside scar only).
# -----------------------------------------------------------

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio import features
from shapely.ops import unary_union
from dbfread import DBF
from affine import Affine
# -----------------------
# User inputs
# -----------------------
lf24_tif   = r"C:\Users\bsf31\Documents\data\NL060\landfire_meszxc7dfpgmqh\LF2024_FBFM40_250_CONUS\LC24_F40_250.tif"  # 2024 FBFM40 (numeric VALUE)
lf23_tif   = r"C:\Users\bsf31\Documents\data\NL060\LFV2023\LF2023_FBFM40_240_CONUS\LC23_F40_240.tif"   # 2023 FBFM40 (numeric VALUE)
dbf_path   = r"C:\Users\bsf31\Documents\data\NL060\landfire_meszxc7dfpgmqh\LF2024_FBFM40_250_CONUS\LF24_F40_250.dbf"   # DBF with VALUE↔MODEL (FBFM40) lookup
scar_path  = r"C:\Users\bsf31\Documents\data\NL060\fire_scar_training_regions.gpkg"  # burn scar polygon
scar_layer = "lake2024_5070"                        # layer name in that gpkg (or None for shapefile)
ring_buffer_m = 1000                            # ring width in meters
out_tif    = r"C:\Users\bsf31\Documents\data\NL060\outputs\LF2024_FBFM40_remap_lakefire.tif"

# -----------------------
# FBFM40 meta (family + climate + summary)
# -----------------------
meta = {
    # GRASS
    "GR1": ("Grass", "Arid–semiarid (EMC 15%)", "Short, patchy, possibly grazed; spread moderate; flame low."),
    "GR2": ("Grass", "Arid–semiarid (EMC 15%)", "Moderately coarse, ~1 ft; spread high; flame moderate."),
    "GR4": ("Grass", "Arid–semiarid (EMC 15%)", "Moderately coarse, ~2 ft; spread very high; flame high."),
    "GR7": ("Grass", "Arid–semiarid (EMC 15%)", "Moderately coarse, ~3 ft; spread very high; flame very high."),
    "GR3": ("Grass", "Subhumid–humid (EMC 30–40%)", "Very coarse, ~2 ft; spread high; flame moderate."),
    "GR5": ("Grass", "Subhumid–humid (EMC 30–40%)", "Dense, coarse, 1–2 ft; spread very high; flame high."),
    "GR6": ("Grass", "Subhumid–humid (EMC 30–40%)", "Dryland grass 1–2 ft; spread very high; flame very high."),
    "GR8": ("Grass", "Subhumid–humid (EMC 30–40%)", "Heavy, coarse, 3–5 ft; spread very high; flame very high."),
    "GR9": ("Grass", "Subhumid–humid (EMC 30–40%)", "Very heavy, coarse, 5–8 ft; spread extreme; flame extreme."),
    # GRASS-SHRUB
    "GS1": ("Grass-Shrub", "Arid–semiarid (EMC 15%)", "Shrubs ~1 ft, low grass; spread moderate; flame low."),
    "GS2": ("Grass-Shrub", "Arid–semiarid (EMC 15%)", "Shrubs 1–3 ft, moderate grass; spread high; flame moderate."),
    "GS3": ("Grass-Shrub", "Subhumid–humid (EMC 30–40%)", "Moderate grass/shrub <2 ft; spread high; flame moderate."),
    "GS4": ("Grass-Shrub", "Subhumid–humid (EMC 30–40%)", "Heavy grass/shrub >2 ft; spread high; flame very high."),
    # SHRUB
    "SH1": ("Shrub", "Arid–semiarid (EMC 15%)", "Low shrub load ~1 ft; spread very low; flame very low."),
    "SH2": ("Shrub", "Arid–semiarid (EMC 15%)", "Moderate load ~1 ft; no grass; spread low; flame low."),
    "SH5": ("Shrub", "Arid–semiarid (EMC 15%)", "Heavy shrubs 4–6 ft; spread very high; flame very high."),
    "SH7": ("Shrub", "Arid–semiarid (EMC 15%)", "Very heavy shrubs 4–6 ft; spread high; flame very high."),
    "SH3": ("Shrub", "Subhumid–humid (EMC 30–40%)", "Moderate shrubs (maybe pine/herb); 2–3 ft; spread low; flame low."),
    "SH4": ("Shrub", "Subhumid–humid (EMC 30–40%)", "Low–moderate shrubs/litter (~3 ft); spread high; flame moderate."),
    "SH6": ("Shrub", "Subhumid–humid (EMC 30–40%)", "Dense shrubs, little/no herb; ~2 ft; spread high; flame high."),
    "SH8": ("Shrub", "Subhumid–humid (EMC 30–40%)", "Dense shrubs, ~3 ft; spread high; flame high."),
    "SH9": ("Shrub", "Subhumid–humid (EMC 30–40%)", "Dense, fine-branched shrubs, 4–6 ft; spread high; flame very high."),
    # TIMBER-UNDERSTORY
    "TU1": ("Timber-Understory", "Semiarid–subhumid (EMC 20%)", "Low grass/shrub + litter; spread low; flame low."),
    "TU2": ("Timber-Understory", "Humid (EMC 30%)", "Moderate litter with shrubs; spread moderate; flame low."),
    "TU3": ("Timber-Understory", "Humid (EMC 30%)", "Moderate litter + grass/shrubs; spread high; flame moderate."),
    "TU4": ("Timber-Understory", "Semiarid–subhumid (EMC 20%)", "Short conifers w/ grass/moss; spread moderate; flame moderate."),
    "TU5": ("Timber-Understory", "Semiarid–subhumid (EMC 20%)", "High conifer litter + shrubs; spread moderate; flame moderate."),
    # TIMBER LITTER
    "TL1": ("Timber Litter", "Recently burned", "Light–moderate load, 1–2 in; spread very low; flame very low."),
    "TL2": ("Timber Litter", "Broadleaf litter", "Low load, compact; spread very low; flame very low."),
    "TL3": ("Timber Litter", "Other conifer litter", "Moderate conifer litter; spread very low; flame low."),
    "TL4": ("Timber Litter", "Mixed fine & coarse", "Moderate load incl. small logs; spread low; flame low."),
    "TL5": ("Timber Litter", "Conifer litter", "High load; light slash/mortality; spread low; flame low."),
    "TL6": ("Timber Litter", "Broadleaf litter", "Moderate load, less compact; spread moderate; flame low."),
    "TL7": ("Timber Litter", "Mixed fine & coarse", "Heavy load incl. larger logs; spread low; flame low."),
    "TL8": ("Timber Litter", "Long-needle pine", "Moderate load/compact; some herb; spread moderate; flame low."),
    "TL9": ("Timber Litter", "Broadleaf / needle drape", "Very high load; spread moderate; flame moderate."),
    # SLASH/BLOWDOWN
    "SB1": ("Slash-Blowdown", "Activity fuel", "10–20 t/ac; 1–3 in class; <1 ft; spread moderate; flame low."),
    "SB2": ("Slash-Blowdown", "Activity fuel", "7–12 t/ac; even 0–3 in; ~1 ft; spread moderate; flame moderate."),
    "SB3": ("Slash-Blowdown", "Activity fuel", "7–12 t/ac; weighted <0.25 in; >1 ft; spread high; flame high."),
    "SB4": ("Slash-Blowdown", "Blowdown (total)", "Total blowdown; not compacted; foliage attached; spread very high; flame very high."),
    # NONBURNABLE
    "NB1": ("Nonburnable", "—", "Urban/suburban"),
    "NB2": ("Nonburnable", "—", "Snow/ice"),
    "NB3": ("Nonburnable", "—", "Agricultural maintained nonburnable"),
    "NB8": ("Nonburnable", "—", "Open water"),
    "NB9": ("Nonburnable", "—", "Bare ground"),
}
fbfm40_meta = (
    pd.DataFrame.from_dict(meta, orient="index", columns=["general_type","climate","summary"])
      .reset_index().rename(columns={"index":"MODEL"})
)

# -----------------------
# Explicit per-pixel overrides (final, corrected)
# dict[(MODEL23, MODEL24)] = TARGET_MODEL
# -----------------------
OVERRIDES = {
    ("SH5","GS1"): "SH1",
    ("SH5","GS2"): "SH2",
    ("SH4","GS2"): "SH3",
    ("SH4","GS1"): "SH3",
    ("GS2","SH2"): "SH2",
    ("GS2","TL1"): "TL6",   # Marc override via explicit table
    ("GS2","GR1"): "GS1",
    ("TU5","TL1"): "TL6",   # Marc override (TL1→TL6)
    ("TU5","TL6"): "TU2",   # TL descriptor match (moderate/low)
    ("GR3","GS1"): "GS3",   # humid grass -> humid GS
    ("GS1","GR1"): "GS1",
    ("GS1","SH2"): "SH2",
    ("GS3","SH1"): "SH2",   # conservative upgrade for humid GS
    ("NB1","SH4"): "NB1",   # NB frozen
    ("TU2","TL6"): "TU1",
    ("SH2","TL3"): "SH2",
    ("SH7","GS2"): "SH2",
    ("TU2","TL1"): "TL6",   # Marc override (TL1→TL6)
    ("NB1","GS2"): "NB1",
    ("TU2","TL3"): "TU1",
}

In [2]:
# -----------------------
# Helpers
# -----------------------
def read_dbf_lut(dbf_path):
    tbl = DBF(dbf_path, load=True, char_decode_errors='ignore')
    df = pd.DataFrame(iter(tbl))
    # Expect columns VALUE (int) and FBFM40 (model code)
    if "FBFM40" in df.columns and "VALUE" in df.columns:
        lut = df[["VALUE","FBFM40"]].rename(columns={"FBFM40":"MODEL"})
    else:
        raise ValueError("DBF must contain VALUE and FBFM40 columns.")
    lut["MODEL"] = lut["MODEL"].astype(str)
    return lut

def family_of(model: str) -> str:
    m = str(model).upper()
    return fbfm40_meta.set_index("MODEL").loc[m, "general_type"] if m in fbfm40_meta["MODEL"].values else None

def climate_of(model: str) -> str:
    m = str(model).upper()
    return fbfm40_meta.set_index("MODEL").loc[m, "climate"] if m in fbfm40_meta["MODEL"].values else None

def to_model(values: np.ndarray, value_to_model: dict) -> np.ndarray:
    # map numeric VALUE -> MODEL string; unknowns -> None
    flat = values.reshape(-1)
    out = np.empty(flat.shape, dtype=object)
    for i, v in enumerate(flat):
        out[i] = value_to_model.get(int(v), None) if v is not None else None
    return out.reshape(values.shape)

def build_mask(gdf: gpd.GeoDataFrame, ds: rasterio.DatasetReader, invert=False):
    # rasterize geometry to mask (True where inside geometry)
    geoms = [geom for geom in gdf.geometry if geom is not None]
    mask = features.geometry_mask(
        geoms,
        out_shape=(ds.height, ds.width),
        transform=ds.transform,
        invert=not invert
    )
    return mask

def reproject_like(gdf: gpd.GeoDataFrame, crs):
    if gdf.crs != crs:
        return gdf.to_crs(crs)
    return gdf

def ring_from_scar(scar_gdf: gpd.GeoDataFrame, buffer_m: float) -> gpd.GeoDataFrame:
    # dissolve scar into single geometry, buffer, difference
    geom = unary_union(scar_gdf.geometry)
    buf  = geom.buffer(buffer_m)
    ring = buf.difference(geom)
    return gpd.GeoDataFrame(geometry=[ring], crs=scar_gdf.crs)

def ring_modals_from_pixels(model_arr, mask_ring, lut_df, fbfm40_meta):
    """Compute modal MODEL per (general_type, climate) and per general_type in the RING."""
    vals = model_arr[mask_ring]  # array of MODEL strings (or None)
    vals = pd.Series(vals, name="MODEL").dropna()
    if vals.empty:
        return {}, {}, {}
    # Join to family + climate
    meta_df = fbfm40_meta.set_index("MODEL").loc[vals.unique()].reset_index()
    fam_map   = dict(zip(meta_df["MODEL"], meta_df["general_type"]))
    clim_map  = dict(zip(meta_df["MODEL"], meta_df["climate"]))
    df = vals.to_frame()
    df["general_type"] = df["MODEL"].map(fam_map)
    df["climate"]      = df["MODEL"].map(clim_map)

    # Modal per (general_type, climate)
    g = (df.groupby(["general_type","climate","MODEL"]).size()
            .reset_index(name="count"))
    # keep argmax per (general_type, climate)
    idx = g.groupby(["general_type","climate"])["count"].idxmax()
    modal_tc = g.loc[idx, ["general_type","climate","MODEL"]]
    ring_modal_tc = { (r.general_type, r.climate): r.MODEL for _, r in modal_tc.iterrows() }

    # Modal per general_type only
    g2 = (df.groupby(["general_type","MODEL"]).size()
            .reset_index(name="count"))
    idx2 = g2.groupby(["general_type"])["count"].idxmax()
    modal_t = g2.loc[idx2, ["general_type","MODEL"]]
    ring_modal_type = { r.general_type: r.MODEL for _, r in modal_t.iterrows() }

    # Canonical VALUE per MODEL in ring
    # (in most Landfire LUTs MODEL↔VALUE is 1–1; this is just being robust)
    # Build a dict VALUE->MODEL and MODEL->VALUE from lut_df:
    model_to_value = dict(zip(lut_df["MODEL"], lut_df["VALUE"]))
    return ring_modal_tc, ring_modal_type, model_to_value

In [3]:
# -----------------------
# Load data
# -----------------------
lut_df = read_dbf_lut(dbf_path)
value_to_model = dict(zip(lut_df["VALUE"], lut_df["MODEL"]))
model_to_value = dict(zip(lut_df["MODEL"], lut_df["VALUE"]))

if scar_layer:
    scar_gdf = gpd.read_file(scar_path, layer=scar_layer)
else:
    scar_gdf = gpd.read_file(scar_path)

with rasterio.open(lf24_tif) as ds24, rasterio.open(lf23_tif) as ds23:
    if (ds24.width, ds24.height, ds24.crs, ds24.transform) != (ds23.width, ds23.height, ds23.crs, ds23.transform):
        raise ValueError("LF2023 and LF2024 rasters must be on the same grid (size/CRS/transform).")

    # align geometries to raster CRS
    scar_gdf = reproject_like(scar_gdf, ds24.crs)
    ring_gdf = ring_from_scar(scar_gdf, ring_buffer_m)
    ring_gdf = reproject_like(ring_gdf, ds24.crs)

    # read arrays
    nodata = ds24.nodata
    v24 = ds24.read(1)  # VALUE 2024
    v23 = ds23.read(1)  # VALUE 2023

    # masks
    scar_mask = build_mask(scar_gdf, ds24)        # True inside scar
    ring_mask = build_mask(ring_gdf, ds24)        # True inside ring

# map VALUE -> MODEL strings
m24 = to_model(v24, value_to_model)
m23 = to_model(v23, value_to_model)

In [4]:
# -----------------------
# Build ring modal dictionaries
# -----------------------
ring_modal_tc, ring_modal_type, model_to_value_ring = ring_modals_from_pixels(m24, ring_mask, lut_df, fbfm40_meta)


In [5]:
# -----------------------
# Apply explicit overrides first (inside scar only)
# -----------------------
inside_idx = np.where(scar_mask.ravel())[0]

# Prepare arrays to hold final MODEL/VALUE (start as copies of 2024)
model_after = m24.ravel().astype(object)
value_after = v24.ravel().copy()

# vector key "m23|m24" to look up overrides
pair_keys = np.char.add(np.where(m23.ravel() == None, "", m23.ravel().astype(str)),
                        np.char.add("|", np.where(m24.ravel() == None, "", m24.ravel().astype(str))))

# Build override dict with "A|B" keys to speed up vectorized map
OV_KEYS = {"%s|%s" % (k[0], k[1]): v for k, v in OVERRIDES.items()}

# apply only inside scar
for idx in inside_idx:
    key = pair_keys[idx]
    tgt = OV_KEYS.get(key, None)
    if tgt is not None:
        model_after[idx] = tgt
        # VALUE from ring's canonical if available, else from LUT
        value_after[idx] = model_to_value_ring.get(tgt, model_to_value.get(tgt, value_after[idx]))

In [6]:
# -----------------------
# Modal + climate fallback for the rest (inside scar, not overridden)
# -----------------------
# Determine which pixels still need fallback: inside scar AND not overridden
overridden_mask = np.zeros(model_after.shape, dtype=bool)
# mark overridden positions (where target model != original m24 OR (pair had explicit mapping))
for idx in inside_idx:
    if OV_KEYS.get(pair_keys[idx], None) is not None:
        overridden_mask[idx] = True

need_fallback = scar_mask.ravel() & (~overridden_mask)

# For fallback we use: ring modal by (general_type, climate) of the CURRENT pixel (from 2024),
# else ring modal by general_type only, else keep m24.
# BUT: keep NB fixed; keep TL as-is EXCEPT TL1 already handled above by explicit rules.

meta_idx = fbfm40_meta.set_index("MODEL")
fam_map  = meta_idx["general_type"].to_dict()
clim_map = meta_idx["climate"].to_dict()

for idx in np.where(need_fallback)[0]:
    cur = model_after[idx] if model_after[idx] is not None else m24.ravel()[idx]
    if cur is None:
        continue
    fam = fam_map.get(cur, None)
    if fam == "Nonburnable":
        # freeze NB
        model_after[idx] = cur
        value_after[idx] = model_to_value.get(cur, value_after[idx])
        continue
    if cur.startswith("TL"):
        # keep TL as-is (TL1 already handled by overrides earlier)
        model_after[idx] = cur
        value_after[idx] = model_to_value.get(cur, value_after[idx])
        continue

    clim = clim_map.get(cur, None)
    # Try ring modal by (family, climate)
    cand = ring_modal_tc.get((fam, clim), None)
    if cand is None:
        # fallback: modal by family only
        cand = ring_modal_type.get(fam, None)
    if cand is None:
        # last resort: keep current
        cand = cur

    model_after[idx] = cand
    value_after[idx] = model_to_value_ring.get(cand, model_to_value.get(cand, value_after[idx]))

# reshape back to raster
model_after = model_after.reshape(v24.shape)
value_after = value_after.reshape(v24.shape)

In [7]:
# -----------------------
# Write output GeoTIFF
# -----------------------
# --- Build an array that has ONLY the updated scar values ---
nodata_out = nodata if (nodata is not None) else -9999  # safe NoData that won’t collide with LF codes
out_arr = np.full(v24.shape, nodata_out, dtype=v24.dtype)
out_arr[scar_mask] = value_after[scar_mask]  # only scar pixels get updated values

# --- Crop to the scar’s tight bounding window (reduces file size) ---
rows, cols = np.where(scar_mask)
if rows.size == 0:
    raise ValueError("Scar mask is empty — nothing to write.")

rmin, rmax = rows.min(), rows.max()
cmin, cmax = cols.min(), cols.max()

out_crop = out_arr[rmin:rmax+1, cmin:cmax+1]
new_transform = ds24.transform * Affine.translation(cmin, rmin)

# --- Write clipped GeoTIFF (only the scar) ---
with rasterio.open(lf24_tif) as src:
    profile = src.profile.copy()

profile.update(
    width=out_crop.shape[1],
    height=out_crop.shape[0],
    transform=new_transform,
    dtype=out_crop.dtype,
    nodata=nodata_out,
    compress="LZW",
)

with rasterio.open(out_tif, "w", **profile) as dst:
    dst.write(out_crop, 1)

print("Wrote scar-only raster:", out_tif)

Wrote scar-only raster: C:\Users\bsf31\Documents\data\NL060\outputs\LF2024_FBFM40_remap_lakefire.tif


In [8]:
# (Optional) also dump a CSV of the overrides you actually applied (for QA)
# Build a quick table of inside-scar pixels where override fired
applied = []
for idx in inside_idx:
    key = pair_keys[idx]
    if OV_KEYS.get(key, None) is not None:
        applied.append((key.split("|")[0], key.split("|")[1], OV_KEYS[key]))
qa_df = pd.DataFrame(applied, columns=["MODEL23","MODEL24","TARGET_MODEL"]).value_counts().reset_index(name="pixels")
# qa_df.to_csv(r"C:\path\to\applied_overrides_summary.csv", index=False)


In [9]:
qa_df

,MODEL23,MODEL24,TARGET_MODEL,pixels
0,SH5,GS1,SH1,30852
1,SH5,GS2,SH2,19154
2,SH4,GS2,SH3,16633
3,SH4,GS1,SH3,12662
4,GS2,SH2,SH2,10840
5,GS2,TL1,TL6,7810
6,GS2,GR1,GS1,5748
7,TU5,TL1,TL6,5376
8,TU5,TL6,TU2,3986
9,GR3,GS1,GS3,1356


In [10]:
# --- Rebuild an upgraded_df_lf from arrays (scar only) ---
scar_idx = np.where(scar_mask)
df_pairs = pd.DataFrame({
    "MODEL_before": m24[scar_idx].astype(object),
    "VALUE_before": v24[scar_idx].astype(int),
    "MODEL_after":  model_after[scar_idx].astype(object),
    "VALUE_after":  value_after[scar_idx].astype(int),
})
# drop rows where model strings are missing
df_pairs = df_pairs.dropna(subset=["MODEL_before","MODEL_after"]).copy()

# count pixels per unique combo (this replaces the old zonal-stats table)
upgraded_df_lf = (
    df_pairs
      .groupby(["MODEL_before","VALUE_before","MODEL_after","VALUE_after"], dropna=False)
      .size()
      .reset_index(name="pixels")
)

# add family/climate labels for convenience (from MODEL_before so Veg Type = scar context)
fam_map  = fbfm40_meta.set_index("MODEL")["general_type"].to_dict()
clim_map = fbfm40_meta.set_index("MODEL")["climate"].to_dict()
upgraded_df_lf["general_type"] = upgraded_df_lf["MODEL_before"].map(fam_map)
upgraded_df_lf["climate"]      = upgraded_df_lf["MODEL_before"].map(clim_map)

# --- SAFETY FREEZE: NB always; TL only when there was NO change (avoid undoing TL1->TL6 etc.) ---
mask_nb = upgraded_df_lf["MODEL_before"].astype(str).str.upper().str.startswith("NB") \
          | upgraded_df_lf["general_type"].eq("Nonburnable")

# TL freeze only if before==after (so explicit TL overrides survive)
is_tl_before = upgraded_df_lf["MODEL_before"].astype(str).str.startswith("TL")
is_tl_after  = upgraded_df_lf["MODEL_after"].astype(str).str.startswith("TL")
mask_tl_nochange = is_tl_before & is_tl_after & (upgraded_df_lf["MODEL_before"] == upgraded_df_lf["MODEL_after"])

freeze_mask = mask_nb | mask_tl_nochange
upgraded_df_lf.loc[freeze_mask, ["MODEL_after","VALUE_after"]] = \
    upgraded_df_lf.loc[freeze_mask, ["MODEL_before","VALUE_before"]].values

# --- Paste-ready table in Marc’s layout ---
veg_label_map = {
    "Shrub": "Chaparral Shrubland",
    "Timber-Understory": "Woodland (Drainages)",
    "Grass": "Grassland",
    "Grass-Shrub": "Grass–Shrub",
    "Timber Litter": "Timber Litter",
    "Slash-Blowdown": "Slash/Blowdown",
    "Nonburnable": "Nonburnable",
}
reference_label   = "Adjacent unburned 1 km buffer (>20 yr)"
sample_area_label = "Lake Fire 2024"   # <- change per notebook/fire

sheet_df = upgraded_df_lf.copy()
sheet_df["Vegetation Type"]       = sheet_df["general_type"].map(veg_label_map).fillna(sheet_df["general_type"])
sheet_df["Sample Number"]         = sheet_df["pixels"].astype(int)
sheet_df["Sample Area (scar)"]    = sample_area_label
sheet_df["LF24 model (scar)"]     = sheet_df["MODEL_before"]
sheet_df["Raster code (scar)"]    = sheet_df["VALUE_before"].astype("Int64")
sheet_df["Reference Sample Area"] = reference_label
sheet_df["LF24 model (ref)"]      = sheet_df["MODEL_after"]
sheet_df["Raster code (ref)"]     = sheet_df["VALUE_after"].astype("Int64")

sheet_df = sheet_df[[
    "Vegetation Type",
    "Sample Number",
    "Sample Area (scar)",
    "LF24 model (scar)",
    "Raster code (scar)",
    "Reference Sample Area",
    "LF24 model (ref)",
    "Raster code (ref)",
]].sort_values(["Vegetation Type","LF24 model (scar)"]).reset_index(drop=True)


In [11]:
sheet_df

,Vegetation Type,Sample Number,Sample Area (scar),LF24 model (scar),Raster code (scar),Reference Sample Area,LF24 model (ref),Raster code (ref)
0,Chaparral Shrubland,769,Lake Fire 2024,SH1,141,Adjacent unburned 1 km buffer (>20 yr),SH2,142
1,Chaparral Shrubland,12622,Lake Fire 2024,SH1,141,Adjacent unburned 1 km buffer (>20 yr),SH5,145
2,Chaparral Shrubland,11664,Lake Fire 2024,SH2,142,Adjacent unburned 1 km buffer (>20 yr),SH2,142
3,Chaparral Shrubland,1435,Lake Fire 2024,SH2,142,Adjacent unburned 1 km buffer (>20 yr),SH5,145
4,Chaparral Shrubland,611,Lake Fire 2024,SH4,144,Adjacent unburned 1 km buffer (>20 yr),NB1,91
5,Chaparral Shrubland,2,Lake Fire 2024,SH4,144,Adjacent unburned 1 km buffer (>20 yr),SH4,144
6,Chaparral Shrubland,96,Lake Fire 2024,SH5,145,Adjacent unburned 1 km buffer (>20 yr),SH5,145
7,Grassland,10963,Lake Fire 2024,GR1,101,Adjacent unburned 1 km buffer (>20 yr),GR2,102
8,Grassland,6973,Lake Fire 2024,GR1,101,Adjacent unburned 1 km buffer (>20 yr),GS1,121
9,Grassland,3496,Lake Fire 2024,GR2,102,Adjacent unburned 1 km buffer (>20 yr),GR2,102
